<a href="https://colab.research.google.com/github/JZ76/f1tenth_simulator_two_agents/blob/master/simpleRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [17]:
pip install -U numpy==1.18.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import tensorflow
print(tensorflow.__version__)

2.8.2


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from tensorflow.python.keras.models import Sequential, Model, load_model
from tensorflow.python.keras.layers import LSTM, Dense, Input, CuDNNLSTM, Masking, Embedding, SimpleRNN, concatenate

import os
import re
import gc

In [3]:
def loading_data(folder):
    check_name = re.compile('^ML')
    datasets = []
    for filename in sorted(os.listdir(folder)):
        files = os.path.join(folder, filename)
        if re.match(check_name, filename) and os.path.isfile(files):
            temp = pd.read_csv(files, skiprows=1, header=None).iloc[:, :-1]
            temp[0] = temp[0].map(lambda t : t/16.0)
            temp[1] = temp[1].map(lambda t : t/0.192)
            datasets.append(np.array(temp, dtype=float))

    gc.collect()
    return datasets

In [4]:
def loading_speed_steering_data(folder):
    check_name = re.compile('^car')
    datasets = []
    for filename in sorted(os.listdir(folder)):
        files = os.path.join(folder, filename)
        if re.match(check_name, filename) and os.path.isfile(files):
            datasets.append(np.array(pd.read_csv(files).iloc[:, [3, 5]], dtype=float))

    gc.collect()
    return datasets

In [5]:
folder = "/content/drive/MyDrive/datasets"
datasets = loading_data(folder)
speed_steering = loading_speed_steering_data(folder)

In [6]:
training_data = datasets

In [7]:
speed_steering_train = speed_steering

In [8]:
X = []
y = []
max_seq_len = 0
for x in training_data:
    X.append(x[:, 2:])
    y.append(x[:, 0:2])
    max_seq_len = max(max_seq_len, x.shape[0])

In [9]:
special_value = -100.0

In [10]:
Xpad = tensorflow.keras.preprocessing.sequence.pad_sequences(
    X, padding="post", value=special_value
)

In [11]:
ypad = tensorflow.keras.preprocessing.sequence.pad_sequences(
    y, padding="post", value=special_value
)

In [ ]:
Xpad = []
ypad = []
for i in range(100):
    Xpad.append(np.full((max_seq_len, 1081), fill_value=special_value))
    ypad.append(np.full((max_seq_len, 2), fill_value=special_value))

for s, x in enumerate(X):
    seq_len = x.shape[0]
    Xpad[s][0:seq_len, :] = x

for s, x in enumerate(y):
    seq_len = x.shape[0]
    ypad[s][0:seq_len, :] = x

In [12]:
training_data = []
X = []
y = []
gc.collect()


102

In [ ]:
Xpad = np.asarray(Xpad)
ypad = np.asarray(ypad)


In [13]:
padded_inputs = tensorflow.keras.preprocessing.sequence.pad_sequences(
    speed_steering_train, padding="post", value=special_value
)

In [ ]:
# LSTM_model = Sequential([
#         Masking(input_shape=(max_seq_len, 1081), mask_value=special_value),
#         SimpleRNN(50, return_sequences=True, input_shape=(None, 1081)),
#         Input(),
#         Dense(64, activation="relu")
#         Dense(32, activation="sigmoid")
#         Dense(2)
# ])
inputA = Input(shape=(None, 1081))
A = Masking(input_shape=(None, 1081), mask_value=special_value)(inputA)
inputB = Input(shape=(None, 2))
B = Masking(input_shape=(None, 2), mask_value=special_value)(inputB)

x = Dense(500, activation="relu")(A)
x = SimpleRNN(400, return_sequences=True, input_shape=(None, 1081))(x)

# x = Dense(200, activation="tanh")(A)
# m = Model(inputs=inputA, outputs=x)

combined = concatenate([x, B], axis=2)
z = Dense(150, activation="selu")(combined)
# z = Dense(64, activation="tanh")(z)
z = Dense(32, activation="tanh")(z)
z = Dense(2, activation="tanh")(z)
LSTM_model = Model(inputs=[inputA, inputB], outputs=z)

In [18]:
LSTM_model = load_model("/content/drive/MyDrive/models/model_RNN_13")

In [19]:
LSTM_model.compile(loss="mean_squared_error", optimizer="RMSprop", metrics=['mean_squared_error'])
LSTM_model.summary()
LSTM_model.fit([Xpad, padded_inputs], ypad, epochs=10, batch_size=5)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 1081)] 0                                            
__________________________________________________________________________________________________
masking (Masking)               (None, None, 1081)   0           input_1[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, None, 500)    541000      masking[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 2)]    0                                            
______________________________________________________________________________________________

In [ ]:
testing_X = []
testing_y = []

for x in testing_data:
    testing_X.append(x[:, 2:])
    testing_y.append(x[:, 0:2])
    

In [ ]:
padded_inputs_X = tensorflow.keras.preprocessing.sequence.pad_sequences(
    testing_X, padding="post", value=special_value
)

In [ ]:
padded_outputs_y = tensorflow.keras.preprocessing.sequence.pad_sequences(
    testing_y, padding="post", value=special_value
)

In [ ]:
testing_data = []
testing_X = []
testing_y = []
gc.collect()

1210

In [ ]:
LSTM_model.evaluate([padded_inputs_X, padded_inputs_test], padded_outputs_y)

2/2 [==============================] - 1s 43ms/step - loss: 0.0288 - mean_absolute_error: 0.0875


[0.02878500334918499, 0.08747684210538864]

In [20]:
LSTM_model.save("/content/drive/MyDrive/new_model_RNN_1")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/new_model_RNN_1/assets
